# 概要

* このコンペで行った内容を共有します．
* 提出したモデルには複数のKernelを使用しており，コードももう少し複雑です．
* Kaggleの紹介と合わせて短時間で共有するため，完結に実行できるレベルにコードを略してまとめています．

# インポートとファイル確認

* test
  * zipファイルで，拡張機能によって中のファイルを直接読み込めます．
* train.csv
  * 学習用に容易された巨大な時系列ファイルです．長さは6億行余り．
* sample_submission.csv
  * 提出用フォーマットのCSVファイルです．

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import os, time, gc
from tqdm import tqdm_notebook

import librosa
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

os.listdir('../input')

# 読込

* pandasという分析ライブラリを使用して学習データを読み込みます
* 空きメモリに余裕が無くなるため，実際はpandasの機能で1500万ずつ分割して読み込んでいます．
* 列の説明
  * acoustic_data -> 音響列．16bitで読まないとメモリエラー
  * time_to_feailure -> 地震までの時間がsec単位で入っています
* 読み込むのに約2分半かかります

In [ ]:
%%time

train = pd.read_csv(
    '../input/train.csv', 
    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
print('loaded:', train.shape)

In [ ]:
train.head()

# 波形を確認

* サンプリングしないとグラフでメモリオーバー．
* オレンジ波形の大きい揺れが地震で，おおよそ地震のタイミングで青い線が0になっています．
* たった16回の地震から次の地震を予測するのは難しそうです，，

In [ ]:
data = train['acoustic_data'].values[::200]
ttf = train['time_to_failure'].values[::200]

fig, ax = plt.subplots(1, 1, figsize=(16, 8))
ax.plot(data, label='acoustic_data')
ax.plot(ttf * 100, label='time_to_feailure')
plt.legend()
plt.show()

fig, ax = plt.subplots(1, 1, figsize=(16, 8))
ax.plot(data[:200000], label='acoustic_data')
ax.plot(ttf[:200000] * 100, label='time_to_feailure')
plt.legend()
plt.show()

del data, ttf
gc.collect()

# 特徴作成

* 他の方のKernelsを参考しつつ，150,000ごとの波形データから特徴を作成していきました．
* 9つの特徴だけに省略していますが，実際は沢山の特徴を作っています．
* 実際の特徴量は下の「算出した特徴量」を参照してください．
* ↓に挙げた以外にも色々な組み合わせを試して，数千個の中から効果がありそうなものを組み合わせて700個程度に絞りました．
* 大体ライブラリを使って容易に計算できますが，そこから過学習しない良い特徴量を選ぶのが大変でした．
* 各特徴量の詳細を学んでいる時間がなく，ライブラリをただ使っただけの特徴も多いです．
* データ追加とカラム追加を同じメソッドにすることで，特徴を増減させた時のコード修正量を減らしています．
* パフォーマンスがあまり変わらなかったので，特徴量をまとめるところはnumpy配列ではなくpythonのlistを使用しています．

----

### 算出した特徴量

* 基本統計量
  * 平均，標準偏差，尖度，幅，分位数，四分位範囲，ピーク数
  * 単純移動平均，単純移動標準偏差，指数移動平均，指数移動標準偏差（各統計量を算出）
* 周波数変換
  * フーリエ変換の実数部・虚数部（特徴がある部分以外をカットして移動平均，移動偏差×各統計量を算出）
  * MFCC：メル周波数ケプストラム係数（2次元データの特徴がある部分の平均値と各統計量を算出）
  * PSD:パワースペクトル密度推定（パラメータ変えて各統計量を算出）
* その他統計量
  * STALTA（地震予測によく使用される古典的なアルゴリズム）
  * 歪度、平均絶対偏差，偏回帰係数，調和平均，幾何平均，基本統計量のパラメータ変更色々
  * 自己相関，歪度の棄却限界値，モーメント，k統計量とその分散
  * Time reversal asymmetry statistic（時系列向け特徴抽出アルゴリズムらしい）
* 更にデータを高周波域を削除して逆変換したものを再度↑の統計量を算出したもの

左5万とか右5万を使った統計量算出は，過学習を起こすだけであまり意味が無さそうだったので採用しませんでした．

In [ ]:
def add(row, key, value, mode=False):
    if mode:
        row.append(key)
    else:
        row.append(value)

def agg(row, d, prefix='', mode=False):
    add(row, 'mean{}'.format(prefix), d.mean(), mode)
    add(row, 'std{}'.format(prefix), d.std(), mode)
    add(row, 'kurt{}'.format(prefix), d.kurt(), mode)
    add(row, 'range{}'.format(prefix), d.max() - d.min(), mode)

    add(row, 'q0.01{}'.format(prefix), np.quantile(d, 0.01), mode)
    add(row, 'q0.05{}'.format(prefix), np.quantile(d, 0.05), mode)    
    add(row, 'q0.95{}'.format(prefix), np.quantile(d, 0.95), mode)
    add(row, 'q0.99{}'.format(prefix), np.quantile(d, 0.99), mode)
    
    add(row, 'iqr{}'.format(prefix), np.subtract(*np.percentile(d, (75, 25))), mode)

def wave2row(d, mode=False):
    row = []

    agg(row, d, mode=mode)
    
    return row

def make_cols():
    cols = wave2row(pd.Series(list(range(150000))), mode=True)
    cols.extend(['seg_id', 'time_to_failure'])
    return cols

## 例1 FFT

* 省いたコードの一例としてフーリエ変換後の実数部のグラフを示します．
* 左右対称なので左半分のみ，更にここから意味のありそうな左から20000個を使っています．

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(np.real(np.fft.fft(train[0:150000]['acoustic_data']))[1:75000])
plt.show()

## 例2 MFCC

* メル周波数ケプストラム係数は有用な特徴量になりました．
* こちらも意味のありそうな領域に絞って使用しています．
* この画像による深層学習（ディープラーニング）も試しましたが，時間がかかり過ぎるのと調整が難しかったので諦めました．

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(librosa.feature.mfcc(
    train[0:150000]['acoustic_data'].values.astype(np.float32), n_mfcc=64)[2:])
plt.show()

# 学習データの特徴量作成

* 150,000毎の特徴（説明変数）と地震までの秒数（目的変数）の表示を作成します．
* 波形情報から扱いやすい行列情報に変換されます．
* tqdmというライブラリを使うと，イテレーター部分をtqdm_notebookで囲むだけで進捗を表示してくれて便利です．

In [ ]:
def make_train(i, f):
    data = f[i:i + 150000]
    
    row = wave2row(data['acoustic_data'])
    
    add(row, 'seg_id', str(i))
    add(row, 'time_to_failure', data[-1:]['time_to_failure'].values[0])
    
    return row

indexes = [i for i in range(0, len(train), 150000)]
train_rows = []
for i in tqdm_notebook(indexes):
    train_rows.append(make_train(i, train))

train_rows = pd.DataFrame(train_rows, columns=make_cols())
train_rows.to_csv('train.csv', index=False)
print('saved train:', train_rows.shape)

train_rows.head()

# テストデータの特徴量作成

* テストデータも同様に特徴量を作成します．
* こちらは150,000毎にファイルが分かれているので1ファイルずつ処理します．
* 最初マルチプロセスで処理していたのですが，マルチプロセスに対応していないライブラリがあったので諦めました．

In [ ]:
def make_test(seg_id):
    data = pd.read_csv('../input/test/' + seg_id + '.csv', dtype={'acoustic_data': np.int16})
    row = wave2row(data['acoustic_data'])
    add(row, 'seg_id', seg_id)
    add(row, 'time_to_failure', 0.0)
    
    return row

submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
test_rows = []
for seg_id in tqdm_notebook(submission.index):
    test_rows.append(make_test(seg_id))

test_rows = pd.DataFrame(test_rows, columns=make_cols())
test_rows.to_csv('test.csv', index=False)
print('saved test', test_rows.shape)

test_rows.head()

# 学習と推論

* 学習データと検証データを組み替えて学習する，クロスバリデーションで学習と推論を行っています．
* 実際はiterationsは2048，他あまりチューニングはしてません
* また，以下のようなロジックで特徴選択を繰り返しました．
  * randam stateを4つ分学習を繰り返して平均を取る
  * 何度か学習を繰り返して，各説明変数の重要度から200程度の特徴に絞る
  * 特徴を10個程度のブロックに分けて，ブロックごとに処理（8, 16, 2４, ３２みたいな可変長ブロック）
  * 特徴を1つずつ増やしていって精度が上がったら追加，下がったら追加しない
  * 特徴を1つずつ減らしていって精度が上がったら削除，下がったら削除しない
  * ブロック分繰り返す

In [ ]:
def learn(params):
    local_train = train_rows.copy()
    local_test = test_rows.copy()
    
    y = local_train['time_to_failure']
    x = local_train.drop(['seg_id', 'time_to_failure'], axis=1)
    
    cols = params['cols']
    if len(cols) == 0:
        cols  = list(x.columns)
    print('cols:', len(cols))
    
    cols.sort()

    x = x[cols]
    test_x = local_test.drop(['seg_id', 'time_to_failure'], axis=1)[cols]
    
    x, y = x.values, y.values
    
    fold = KFold(n_splits=5, random_state=params['random_state'], shuffle=True)
    
    val_accs = []
    val_preds = []
    weights = []
    submissions = []
    
    for i, (train_index, val_index) in enumerate(fold.split(x, y)):
        train_x, train_y = x[train_index], y[train_index]
        val_x, val_y = x[val_index], y[val_index]
        
        print('  fold:', i + 1, train_x.shape, val_x.shape)
        
        model = CatBoostRegressor(
            iterations=256, learning_rate=0.015, verbose=32, eval_metric='MAE',
            use_best_model=True, task_type='GPU')
        model.fit(train_x, train_y, eval_set=(val_x, val_y))
        
        val_pred = model.predict(val_x)
        val_pred = np.where(val_pred < 0, 0, val_pred)
        val_acc = mean_absolute_error(val_y, val_pred)
        val_accs.append(val_acc)
        
        val_f = pd.DataFrame()
        val_f['val_index'] = val_index
        val_f['time_to_failure'] = val_pred
        val_preds.append(val_f)

        weight = pd.DataFrame()
        weight['col'] = cols
        weight['weight'] = model.feature_importances_
        weights.append(weight)

        test_pred = model.predict(test_x)
        test_pred = np.where(test_pred < 0, 0, test_pred)
        submission = pd.DataFrame()
        submission['seg_id'] = local_test['seg_id']
        submission['time_to_failure'] = test_pred
        submissions.append(submission)

    print('  ', ['{0:.4f}'.format(acc) for acc in val_accs], '{0:.4f}'.format(np.mean(val_accs)))
    
    weights = pd.concat(weights) if len(weights) > 1 else weights[0]
    val_preds = pd.concat(val_preds) if len(val_preds) > 1 else val_preds[0]
    val_preds.sort_values('val_index', inplace=True)
    val_preds.reset_index(drop=True, inplace=True)
    submissions = pd.concat(submissions) if len(submissions) > 1 else submissions[0]
    
    return submissions, weights, val_accs, val_preds

cols = []
submissions, weights, val_accs, val_preds = learn({ 'cols': cols, 'random_state': 48 })

# 各特徴の重要度

* 各特徴の目的変数に対する貢献を重要度として表示します．
* 数値の絶対的な大きさに意味は無く，差を見ます．
* eli5というライブラリによる算出も試しましたが，処理に時間がかかるので採用せず，，
* 色々試して曖昧な指標だと判断，700 -> 200の選択にだけ使用して，細かい特徴選択には使いませんでした．

In [ ]:
means = weights.groupby('col', as_index=False).mean().rename(columns={'weight':'mean'})
bars = pd.merge(weights, means, on='col', how='left')
bars.sort_values('mean', ascending=False, inplace=True)

bars.to_csv('weight.csv', index=False)
print('saved weight:', bars.shape)

plt.figure(figsize=(8, 4))
sns.barplot(x='weight', y='col', data=bars)
plt.show()

# 特徴の精査

## 目的変数と特徴の関係を確認

* 特徴が想定通りのデータになっているか，確認を行います．
* グラフで目視しながら，特徴を選んだり，調整を加えたりします．

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 4))
ax.plot(train_rows['time_to_failure'])
ax.plot(train_rows['q0.95'])
plt.show()

## 学習データの正解値と予想値を確認

* 正解値と予測値をグラフに表示して確認します
* ９つの簡易的な特徴でも，ある程度推論できていることが分かります．
* グラフにもあるように，地震が長時間起こらないケースが2，３あり，少ない地震からそれを予測するのが難しいです．

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 4))
ax.plot(train_rows['time_to_failure'])
ax.plot(val_preds['time_to_failure'])
plt.show()

## 相関と分布を確認

* 総当たりで各特徴間の相関を表示しています．
* 目的変数と各特徴の相関も参考にします．
* まったく同じ相関や似た相関を持った特徴は一方を除外すると精度が上がることがありました．

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(10,8))

sns.heatmap(
    train_rows.drop('seg_id', axis=1).corr(), linewidths=0.1, vmax=1.0, square=True,
    cmap=colormap, linecolor='white', annot=True)
plt.show()

In [ ]:
sns.pairplot(train_rows.drop('seg_id', axis=1))
plt.show()


# 実際の最適化で選んだ特徴の例

* 他にも学習モデルを作ってるので，これらの特徴グループだけではありません．
* 例えば7番目の特徴は，フーリエ変換虚数部の低周波領域20000の移動平均1000の分位数0.05.
* ewmは指数移動平均，denoiseはハイパスフィルタを通したものです．
----
* mean_mfcc15
* q0.05_ewm-std30 
* mean_mfcc4
* range_mfcc_mean
* q0.05_denoise_roll-std100
* q0.05_fft-imag_low20000_roll-std1000
* q0.05_fft-real_low20000_roll-std1000 
* q0.05_fft-real_low20000_roll-mean1000
* peak-count10_ewm-mean3000
* q0.05_denoise_roll-std10
* spkt50_denoise
* std_mfcc3
* peak-count50_ewm-mean30
* q0.05_ewm-mean3000
* mean_mfcc7
* q0.05_fft-imag_low20000_roll-std100
* q0.05_roll-mean10
* q0.01_denoise_roll-mean10
* q0.2_base
* kurt_mfcc10
* peak-count50_denoise_ewm-std30

# 他のモデルを作成

* ベースはCatBoostという決定木ベースの勾配ブースティングに基づく機械学習ライブラリで最適化しました．
* 他にLightGBM，ニューラルネットワークを用いて別の学習モデル作成しましたが，期間が残り少なく上手く精度を上げることができませんでした．
* 作成した3つのモデルに加えて，カーネルに上がっていた遺伝的アルゴリズムのモデルも使用しています．

# 結果を提出

* 以下はクロスバリデーションを平均化しただけの結果を保存しています．

In [ ]:
subs = submissions.groupby('seg_id', as_index=False).mean()
subs.to_csv('submission.csv', index=False)
print('saved submission', subs.shape)
subs.head()

# 結果の分布を確認

* 正解，予想，最終結果それぞれの結果の分布を確認してみます．
* 今回の結果は分布の違いが大きく見えます..
* 特に10秒以上の長いtime_to_failureを如何に予想するかが，よく議論に挙がっていました．

In [ ]:
sns.distplot(train_rows['time_to_failure'], kde=True, label='train')
sns.distplot(val_preds['time_to_failure'], kde=True, label='val')
sns.distplot(subs['time_to_failure'], kde=True, label='test')
plt.legend()
plt.show()

# 評価

* MAE（Mean Absolute Error：平均絶対誤差）で評価され，リーダーボードにランキング表示されます．

![MAE](https://wikimedia.org/api/rest_v1/media/math/render/svg/3ef87b78a9af65e308cf4aa9acf6f203efbdeded)

### クロスバリデーションによる学習データでの評価

* CatBoostで学習を進めるとどんどん下がっていくものの，テストデータによるランキング（公開LB）が下がってしまうので過学習と判断．
* 過学習していると思われる結果を除くと1.97〜8程度で頭打ちに..

### 公開リーダーボード（一部テストデータによる暫定ランキング）

* シェイクアップ前の公開LB
* 最初の頃からクロスバリデーションでの結果よりかなり低い値になっていました（CV2.0で公開LB1.5とか）
* 学習データをシャッフルして推論すると稀に同じような結果になるので，評価に使われているテストデータの一部がかなり偏っていそう．
* シングルモデルが1.43〜４辺りで頭打ちに．
* 幾つか試すとたまに精度が上がるも，シード値やデータを少しいいじると下がってしまうのでラッキーな過学習だと判断．
* ディスカッションでもランキング（プライベートLB）確定時に激しい順位のシェイクアップが起こりそうとの噂..
* 異なるアプローチで最適化された学習モデルを平均して提出すると精度が上がるらしく，実践してみました．
* 0開始と75,000開始で分割した学習データ２パターン×３モデルで６つの学習モデルを作成．
* 遺伝的アルゴリズムモデル，少し前に作って精度が良かったモデルを合わせて8つの平均値を最終提出．
* ベストモデルの公開LBの評価は「1.38223」で205位．

### プライベートリーダーボード（シェイクアップ後のランキング確定）

* 公開リーダーボードの評価は「2.4514」で77位（モデルを2つ提出できるので，公開LBとは違うモデルがランクイン）
* 提出履歴を眺めて見ると，↑のほうに示したように特徴量を20個程度に絞った学習モデルが高い評価を出していました．
* 提出履歴での最高は「2.4074」で順位にすると21位であるものの，そのモデルの良し悪しをコンペ中に判断できなかったわけで意味無し．

# 上位入賞者のアプローチ

* https://www.kaggle.com/c/LANL-Earthquake-Prediction/discussion
* 勉強が足りず分からないことが多いものの，以下のような検証が必要だった模様．
  * 音響データでの地震発生から時間リセットまでの外れ値の扱い
  * 学習データとテストデータの特徴の分布や予想結果の異なりを検証
    * 敵対的検証
    * 学習データを繰り返しサンプリングしてKS検定
    * 学習データにノイズを加える